# Rerun RGBD camera example
This example shows how to use [rerun](https://www.rerun.io/) for logging images, image annotations, pointclouds, transforms between elements in the world, time-series in a way that you might want to do for a robotic manipulation setup. Make sure you have a **ZED2I camera connected**.


Rerun has more features such as logging meshes, logging 3D bboxes, URDFs (in process). Check the docs to learn more. 
Also note that this tool is still very 'young', it became public only mid february 2023. There are many issues that need to be solved. E.g. the pypi wheel is broken atm and you cannot save the rerun file from the UI. The team is very responsive on github so shoot if you find issues etc.

In [ ]:
from airo_camera_toolkit.cameras.zed2i import Zed2i
import rerun
#autoreload
%load_ext autoreload
%autoreload 2

In [ ]:
# start rerun. If the UI is already running, it will connect to it. Otherwise it will start a new UI and connect to it.
# you can also start rerun using `python -m rerun`.
rerun.init("test",spawn=True)

In [ ]:
zed = Zed2i(depth_mode=Zed2i.NEURAL_DEPTH_MODE, resolution=Zed2i.RESOLUTION_2K)

In [ ]:
pointcloud = zed.get_colored_point_cloud()
rgb = zed.get_rgb_image()
depth = zed.get_depth_image()

In [ ]:
zed.depth_enabled = False

In [ ]:
# log the colored pointcloud to the UI
rerun.log_points("world/camera1/pointcloud",positions=pointcloud[:,:3],colors=pointcloud[:,3:6])


In [ ]:
pointcloud[3156][3:6]

In [ ]:
# log the non-colored pointcloud to the UI in the same world/camera1 space
rerun.log_points("world/camera1/non_color_pointcloud",positions=pointcloud[:,:3],colors=[0.5,0.5,0.5])

In [ ]:
# log the rgb and depth images to the UI in a new image/camera1 space, this will automatically become a 2D image viewer.
rerun.log_image("image/camera1/rgb",rgb)
rerun.log_image("image/camera1/depth",depth)

In [ ]:
# log a dummy 2D bbox 
rerun.log_rect("image/camera1/rect",[20,100,300,500])

In [ ]:
# log some dummy keypoints and attach labels
rerun.log_points("image/camera1/keypoints",[[600,500],[400,500]],keypoint_ids=[0,1],radii=20, labels=["keypoint1","keypoint2"])

In [ ]:
# log dummy camera extrinsics from the world space. 
# This specifies the pose of camera in world.
translation = [0,0.5,0.7] 
rotation = [-0.707,0,0,0.707] #scalar-last! 
rerun.log_rigid3("world/camera1", parent_from_child=(translation, rotation))

In [ ]:
# log the actual camera intrinsics, to create a pinhole camera model in the UI.
width = 1920
height = 1080
intrinsics = zed.intrinsics_matrix
rerun.log_pinhole("world/camera1/rgb",
                  child_from_parent = intrinsics,
                  width = width,
                  height = height)
rerun.log_image("world/camera1/rgb",rgb)


In [ ]:
# set up the 'view' of the 3D world space. This is for convenience so that rerun can sensible starting orientations for the spaces. 
rerun.log_view_coordinates("world", up="+Z", right_handed=True, timeless=True)
rerun.log_view_coordinates("world/camera1", xyz="RDF")


In [ ]:
# log some more data
for _ in range(5):
    pointcloud = zed.get_colored_point_cloud()
    rerun.log_points("world/camera1/pointcloud",positions=pointcloud[:,:3],colors=pointcloud[:,3:6])

In [ ]:
# log time-series (e.g. F/T sensor)
for i in range(100):
    rerun.log_scalar("world/robot/force",i)


In [ ]:
# close the camera
zed.camera.close()